# Introduction to HOOMD-Blue

This is a tutorial for HOOMD-Blue, adapted from the tutorial found in HOOMD-Blue's documentation (https://hoomd-blue.readthedocs.io/en/latest/tutorials.html). 

Use the Introduction to Molecular Dynamics tutorial to help you complete this notebook and run your first simulation. Answer the bolded questions in a new cell underneath the question. 

Once you have completed the questions and ran all the cells in the notebook successfully, select File --> Print Preview to open the print preview. Then using Ctrl+P (or Cmd+P on Mac) open the print window and save it as a PDF. Submit the PDF to Canvas along with your written homework.

For more information on how to navigate Jupyter Notebooks, visit this webpage: https://jupyter-notebook.readthedocs.io/en/stable/examples/Notebook/Notebook%20Basics.html

## Getting Started

The first step to using HOOMD is to import the package in python, then specify the device that you will be using to execute the simulation operations. For this course, you will always use a CPU device.

In [ ]:
import hoomd #import the hoomd package into python
cpu = hoomd.device.CPU() #specify the CPU device

We also want to import a few other packages so that we can do calculations and make figures.

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')
import itertools

When creating a simulation in HOOMD, you must instantiate a Simulation object and a State object. **What do these objects each contain?** We can instantiate a simulation below. 

In [ ]:
#instantiate a simulation
sim = hoomd.Simulation(device=cpu, seed=1)

A molecular dynamics simulation models the movement of particles over time by integrating Newton's equations of motion numerically, advancing the state of the system forward by time *dt* on each time step. 

Each simulation requires an **integrator** to advance the simulation. We can create one by using the integrator class in hoomd's md package to implement molecular dynamics integration. 

In [ ]:
integrator = hoomd.md.Integrator(dt=0.005)

Now that the integrator has been estabilished, we need to describe the interactions between particles by using the force term of the equations of motion. The default in HOOMD is to have no forces, and you can add as many force objects as you'd like to the integrator to describe particle interaction. 

We are going to establish a pair potential, that describes the potential energy between a single pair of particles given their separation distance *r*. The pair potential we will use is called the Lennard-Jones potential, and 
is given as: $$V_{LJ}(r)=4\epsilon\left[\left(\frac{\sigma}{r}\right)^{12}-\left(\frac{\sigma}{r}\right)^6\right]$$
where $\sigma$ is the diameter of the particle in LJ units, and $\epsilon$ is the energy in LJ units.

In [ ]:
sigma=1
epsilon=1
r = np.linspace(0.95, 3, 500)
V_lj = 4*epsilon*((sigma/r)**12 - (sigma/r)**6)

fig, ax = plt.subplots()
ax.plot(r, V_lj)
ax.set_xlabel('r')
ax.set_ylabel('V')
plt.show()

The Lennard-Jones potential has two main sections. **What are these sections and what do they represent?**

Pair potentials are defined between all pairs of particles. In molecualr dynamics simulations, however, pair potentials are only evaluated for pairs of particles with distance $r<r_{cut}$, where $r_{cut}$ is a user defined cutoff distance. This allows for much faster computation, and is done using a neighbor list. We are going to use a cell based neighbor list that divides the simulation box into smaller boxes (aka cells) to find neighbors within the cutoff distance.

In [ ]:
cell = hoomd.md.nlist.Cell(buffer=0.4)

With an established neighbor list method, we can create the Lennard-Jones pair potential. 

In [ ]:
lj = hoomd.md.pair.LJ(nlist=cell)

Pair potentials in HOOMD need paramters for every pair of particle types in the simulation. In our initial simulation, we will only have one particle type, type 'A'. We can define the A-A particle interaction as follows.

In [ ]:
lj.params[('A', 'A')] = dict(epsilon=1, sigma=1)
lj.r_cut[('A', 'A')] = 2.5

Now we can add the force to our integrator so that it is included in the calculations. 

In [ ]:
integrator.forces.append(lj)

Now that we have selected our integrator and added forces to model particle interactions, we must choose what integration method we would like to use. Integration methods define the equations of motion that will be applied to the particles in the system. For this simulation, we will use a constant volume integration method that will implement Newton's laws while the thermostat scales the particle velocities. 

In [ ]:
nvt = hoomd.md.methods.ConstantVolume(
    filter = hoomd.filter.All(), 
    thermostat=hoomd.md.methods.thermostats.Bussi(kT=1.5))

In the above code block, `filter` is a particle filter that selects which particles the integration method applies to. For this simulation, we've chosen to apply the constant volume integration method to all particles. `kT` is the temperature multipled by the Boltzmann constant, and has units of energy.

**Now, add this method to the integrator.**

In [ ]:
#add the integration method to the integrator. 


# Initializing the System

We are going to initialize a system of Lennard-Jones particles on a cubic lattice with $N=4*m^3$ particles.

In [ ]:
m = 4
N_particles = 4*m**3

In molecular dynamics simulations, particles can theoretically occupy any position within the simulation box, however, because we are using the LJ potential, we must ensure that the particles do not overlap. **Why is this? What would happen if the particles *did* overlap?**

In [ ]:
spacing = 1.3
K = int(np.ceil(N_particles**(1/3)))
L = K*spacing
x = np.linspace(-L/2, L/2, K, endpoint=False)
position = list(itertools.product(x,repeat=3))

Now that we have generated the initial particle positions, we can add them to our simulation state. HOOMD uses GSD files to store information about the simulation box, particle positions, and other properties of the simulation state. We must import the GSD module to create our own GSD file. 

In [ ]:
import gsd.hoomd

The **Frame** object stores the state of the system. 

In [ ]:
frame = gsd.hoomd.Frame()
frame.particles.N = N_particles
frame.particles.position = position[0:N_particles]

What we just did is initialize a frame of our simulation that contains the positions of all particles. Each particle has a type, and each type has a name. We must also include this information in our initial frame.

In [ ]:
frame.particles.typeid = [0]*N_particles
frame.particles.types = ['A']

The final piece of information we need to include in our initialization for this simulation is size of our box. The simulation box that we are creating is a periodic box. **What is the significance of using a periodic box?** 

The box configuration takes in the length of the box in the x, y, and z dimensions as well as the tilt factor used in the box (0 for our simulation).

In [ ]:
frame.configuration.box = [L, L, L, 0, 0, 0]

Finally, we can write this information to our gsd file. We will title it `init_config.gsd`

In [ ]:
with gsd.hoomd.open(name='init_config.gsd', mode='w') as f:
    f.append(frame)
f.close()

## Starting up the Simulation

We have already done a lot of the initial steps that are required to startup a simulation in HOOMD-Blue. To recap, here's what we've done so far: 
```python
cpu = hoomd.device.CPU()
sim = hoomd.Simulation(device=cpu, seed=1)

integrator = hoomd.md.Integrator(dt=0.005)
cell = hoomd.md.nlist.Cell(buffer=0.0)
lj = hoomd.md.pair.LJ(nlist=cell)
lj.params[('A', 'A',)] = dict(epsilon=1, sigma=1)
lj.r_cut[('A', 'A')] = 2.5
integrator.forces.append(lj)
nvt = hoomd.md.methods.ConstantVolume(
    filter=hoomd.filter.All(),
    thermostat=hoomd.md.methods.thermostats.Bussi(kT=1.5))
integrator.methods.append(nvt)
```
Now we need to fill in the gaps so that we can start running our first simulation!


The first thing we need to do is initialize the system state. Typically, this is done right after initializing the simulation (`sim = hoomd.Simulation(device=cpu, seed=1)`). We are going to use the `init_config.gsd` file that we created earlier to initialize our sysem state. 

In [ ]:
sim.create_state_from_gsd(filename='init_config.gsd')

Now we can assign our integrator to the simulation. 

In [ ]:
sim.operations.integrator = integrator

Now that that's all done, we can assign random initial velocities to our particles. The default in HOOMD-Blue is for particles to have zero initial velocity, however it is important that the particles are initialized with non-zero velocities so that the integrator can compute forces properly. We will use the `thermalize_particle_momenta` method to assign Gaussian distributed random initial velocities, and set the velocity center of mass to zero. 

In [ ]:
sim.state.thermalize_particle_momenta(filter=hoomd.filter.All(), kT=1.5)

If we want to be able to access the thermodynamic information about our simulation later, we can include a **Compute**, which is an **Operation** that computes desired properties of a system. We are going to use the **ThermodynamicQuantities** class to compute the thermodynamic properties of the system.

In [ ]:
thermo_properties = hoomd.md.compute.ThermodynamicQuantities(filter=hoomd.filter.All())
sim.operations.computes.append(thermo_properties)

In order to be able to access information about how our system evolves over time, we want to make sure that we log the properties of the simulation. Within HOOMD-Blue there are certain **Loggable quantities** that we can access by creating a **Logger**. 

First, we must create the logger object. This will store a list of all the properties that we are interested in and provide it to our **Writer** when called.

In [ ]:
logger = hoomd.logging.Logger()

Now, we can add the loggable quantaties that we are interested in to our logger object.

In [ ]:
logger.add(thermo_properties) #we can add lists of properties like this
logger.add(sim, quantities=['timestep', 'walltime']) #or we can add individual properties

Now that we've created our Logger, we need to create a **Writer** to write these quantities to a GSD file. We want this writer to write out our logged quantities every 100 timesteps, so we will use a periodic trigger. 

In [ ]:
gsd_writer = hoomd.write.GSD(filename='log.gsd',
                            trigger=hoomd.trigger.Periodic(100),
                            mode='wb',
                            filter=hoomd.filter.All())
sim.operations.writers.append(gsd_writer)

*Note: If we are interested only in the loggable quantities, and not in the trajectory of our particles, we could set `filter=hoomd.filter.Null()`. This will ensure that we do not store any information about the positions of our particles. Since we are interested in the particle trajectory, we have applied this writer to all particles.*

Now we assign our logger to the writer we just created.

In [ ]:
gsd_writer.logger = logger

We could also create a table writer to write out some information while the simulation is running. To do this, we follow similar steps to the ones above.

In [ ]:
table_logger = hoomd.logging.Logger(categories=['scalar', 'string'])
table_logger.add(sim, quantities=['timestep', 'tps', 'walltime'])

table = hoomd.write.Table(trigger=hoomd.trigger.Periodic(period=1000),
                          logger=table_logger)
sim.operations.writers.append(table)

Finally, we can run the simulation! To do this, we simply use `sim.run(timesteps)` and specify the number of timesteps we'd like to run. We can run this simulation for 10,000 timesteps. 

In [ ]:
sim.run(10000)

We are now going to flush the write buffer, which just allows us to access the information we just wrote out within the same notebook. This won't always be necessary.

In [ ]:
gsd_writer.flush()

We can save the final configuration of the system to a GSD file so that we can access it later if necessary. 

In [ ]:
hoomd.write.GSD.write(state=sim.state, filename='final_config.gsd', mode='wb')

## Reading Our Logged Data

GSD files are binary files, which means you can't just open it and read your results like a typical text file. In order to access the information we've logged about our simulation, we must use the GSD module. 

To read the logged data for the simulation state (aka for the whole system and not per-particle), we can use GSD's `read_log` function.

In [ ]:
log_data = gsd.hoomd.read_log('log.gsd')

This will give us a dictionary of logged quantities with keys that describe the class that computed said quantity.

In [ ]:
list(log_data.keys())

For example, if we want to see the potential energy of the system over time, we can access it using the key `log/md/compute/ThermodynamicQuantities/potential_energy`.

In [ ]:
log_data['log/md/compute/ThermodynamicQuantities/potential_energy']

GSD provides logged quantities as an array with one entry per frame. For example, we could plot the potential energy over time using our logged quantities. 

In [ ]:
timestep = log_data['configuration/step']
potential_energy = log_data['log/md/compute/ThermodynamicQuantities/potential_energy']

fig, ax = plt.subplots()
ax.plot(timestep, potential_energy)
ax.set_xlabel('timestep')
ax.set_ylabel('potential energy')
plt.show()

Try plotting the pressure of the system over time in the cell below.

In [ ]:
timestep = 
pressure = 

fig, ax = plt.subplots()
ax.plot()
ax.set_xlabel('')
ax.set_ylabel('')
plt.show()

To access per-particle information from the logger such as the particle positions over time, we can open the gsd file as a trajectory.

In [ ]:
traj = gsd.hoomd.open('log.gsd', mode='r')

Information on the particle positions is stored in `traj`, and can be accessed by frame. If we want the positions of all particles in the last frame, for example, we can access this by calling the last (-1) frame of traj. 

In [ ]:
traj[-1].particles.position

Once you have completed this notebook, visualize your system using Ovito. Load your `log.gsd` file into Ovito and change the color of the particles to your favorite color. Make sure they are the correct size. Create a movie of the simulation and submit the movie along with the other files you are submitting for this homework assignment. 